In [1]:
import numpy as np
import scipy

### Define Hubbard Hamiltonian

In [2]:
from openfermion.utils import HubbardSquareLattice 

x_n = 2 
y_n = 2 
n_dofs = 1
periodic = 0
spinless = 0 

lattice = HubbardSquareLattice(x_n, y_n, n_dofs=n_dofs, periodic=periodic, spinless=spinless)

In [3]:
from openfermion.hamiltonians import FermiHubbardModel
from openfermion.utils import SpinPairs
tunneling = [('neighbor', (0, 0), 1.)] 
interaction = [('onsite', (0, 0), 2., SpinPairs.DIFF)] 
potential = None
mag_field = 0. 
particle_hole_sym = False 

hubbard = FermiHubbardModel(lattice , tunneling_parameters=tunneling, interaction_parameters=interaction, 
                            potential_parameters=potential, magnetic_field=mag_field, 
                            particle_hole_symmetry=particle_hole_sym)

### Find tunneling eigenvectors with highest fidelity with Hubbard ground state

In [4]:
from openfermion.utils import inner_product 
from openfermion.transforms import get_sparse_operator

def overlap(a, b):
    """Calculates the overlap between vectors a and b. This metric is also known as fidelity. """
    inner = inner_product(a, b)
    return (np.conjugate(inner) * inner).real

# Lowest eigenvalue/vector of Hubbard Hamiltonian
# The column v_hub[:, i] is the eigenvector corresponding to the eigenvalue w_hub[i]
hub_sparse = get_sparse_operator(hubbard.hamiltonian())
w_hub, v_hub = scipy.sparse.linalg.eigsh(hub_sparse, k=1, which='SA') 

# From looking at the spectrum,  I noticed the ground state was degenerate. 
# There were 16 eigenvectors with the lowest eigenvalue. 
# Actually, the initial state might not even be a ground state. Let's try 
# as many of the eigenvectors as we can. 
tun_sparse = get_sparse_operator(hubbard.tunneling_terms())
# k can be at most n-2 where n is the dimension of the matrix
w_tun, v_tun = scipy.sparse.linalg.eigsh(tun_sparse, k=np.shape(tun_sparse)[0]-2, which='SA')

# Let's check overlap of each of the ground states. Hopefully, there's an obvious winner. 
for i in range(len(w_tun)):
    fid = overlap(v_hub[:, 0], v_tun[:, i]) 
    if fid > 0.10:
        print('Eigenvector v_tun[:, {}] has ovelap {}.'.format(i, fid))

Eigenvector v_tun[:, 11] has ovelap 0.12622062025255504.
Eigenvector v_tun[:, 19] has ovelap 0.1985620731386243.
Eigenvector v_tun[:, 34] has ovelap 0.10964243588147657.


None of these stand out as the obvious best choice, so we'll perturb tunneling term with the interaction term and hope we have better luck. 

### Find perturbed eigenvectors with highest fidelity with Hubbard ground state

In [5]:
s = 1e-4
int_sparse = get_sparse_operator(hubbard.interaction_terms())
perturbed_sparse = tun_sparse + s * int_sparse

w_per,v_per = scipy.sparse.linalg.eigsh(perturbed_sparse, k=5, which='SA') 

for i in range(len(w_per)):
    fid = overlap(v_hub[:, 0], v_per[:, i])
    print('Eigenvector v_per[:, {}] has overlap {}.'.format(i, fid))
    
# WOW, there's a clear winner here v_per[:, 0]
per_state_most_overlap = v_per[:, 0]

Eigenvector v_per[:, 0] has overlap 0.9770315592604026.
Eigenvector v_per[:, 1] has overlap 3.658337908110008e-20.
Eigenvector v_per[:, 2] has overlap 5.060794393010509e-25.
Eigenvector v_per[:, 3] has overlap 8.521614327774115e-26.
Eigenvector v_per[:, 4] has overlap 2.4103963324261132e-25.


### Find tunneling eigenvector with highest fidelity with best perturbed eigenvector

In [6]:
max_overlap = 0
index_max_overlap = 0

for i in range(len(w_tun)):
    fid = overlap(per_state_most_overlap, v_tun[:, i])
    if fid > max_overlap: 
        max_overlap = fid 
        index_max_overlap = i 
print("Eigenvector v_tun[:, {}] had the maximum overlap of {} with the best perturbed state. \
       It has eigenvalue {}".format(index_max_overlap, max_overlap, w_tun[index_max_overlap]))
initial_state = v_tun[:, index_max_overlap]


Eigenvector v_tun[:, 19] had the maximum overlap of 0.2032311164794801 with the best perturbed state.        It has eigenvalue -4.000000000000009


### Use `prepare_gaussian_state` and `final_wavefunction` to find circuit that produces state with highest fidelity with above tunneling eigenvector

In [7]:
from openfermion.transforms import get_quadratic_hamiltonian 

# Convert to QuadraticHamiltonian instance 
tun_quad = get_quadratic_hamiltonian(hubbard.tunneling_terms())

orbital_energies, constant = tun_quad.orbital_energies()
print(orbital_energies)

[-2.00000000e+00 -2.00000000e+00 -1.91765100e-16  1.64346022e-32
  8.93943564e-17  5.82889146e-16  2.00000000e+00  2.00000000e+00]


In [8]:
# Get all possible combinations of orbital energies which have a sum of -4 
import itertools 
zero_indices = [2, 3, 4, 5] # Indices of 0's in orbital_energies
orbital_energies_combs = [] 
for r in range(len(zero_indices) + 1):
    for subset in itertools.combinations(zero_indices, r):
        # Include indices for both -2's and then indices of some 0's
        orbital_energies_combs.append([0, 1] + list(subset))
        
# Now, generate each eigenvector from a combination of orbital_energies 
# and compare it to our desired tunneling eigenvector. 
# This way we know which orbital_energies combination yields our 
# desired eigenvector. 

# from openfermion.utils import gaussian_state_preparation_circuit
from openfermioncirq import prepare_gaussian_state
from cirq import Circuit, final_wavefunction, LineQubit

overlaps = []
for comb in orbital_energies_combs:
    state = final_wavefunction(Circuit(prepare_gaussian_state(
            LineQubit.range(8), # We have an 8 qubit Hamiltonian 
            tun_quad, 
            occupied_orbitals=comb)))
    overlaps.append(overlap(state, initial_state))

# Find state with top overlap and save the Circuit object that created it 
best_state_index = np.argmax(overlaps)

print("Orbital energies {} resulted in a state with {} overlap with our desired state.".format(
    orbital_energies_combs[best_state_index], overlaps[best_state_index]))

Orbital energies [0, 1] resulted in a state with 0.2032310999224089 overlap with our desired state.


**Shouldn't we have a much higher overlap here? We use `prepare_gaussian_state` to generate ground states of the tunneling term and calculate fidelity with another ground state of the tunneling term?** We should hit an eigenvector with fidelity close to 1, right?

If I instead compute fidelity with the best *perturbed* state, I get an eigenvector with fidelity close to 1.

In [13]:
overlaps = []
for comb in orbital_energies_combs:
    state = final_wavefunction(Circuit(prepare_gaussian_state(
            LineQubit.range(8), # We have an 8 qubit Hamiltonian 
            tun_quad, 
            occupied_orbitals=comb)))
    overlaps.append(overlap(state, per_state_most_overlap))

best_state_index = np.argmax(overlaps)
print("Orbital energies {} resulted in a state with {} overlap with our desired state.".format(
    orbital_energies_combs[best_state_index], overlaps[best_state_index]))

Orbital energies [0, 1] resulted in a state with 0.999999917691482 overlap with our desired state.


This is good, but it seems we just got lucky with the basis that `occupied_orbitals` chose. A degenerate ground state means any vector in the span of the ground states is a valid ground state. Is there any way we can specify the basis that `occupied_orbitals` chooses? Or at least specify linear combinations of the basis vectors? 